Here labels grouped
Used model ResNet18 (pretrained on EmotionNet)

In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import functional as TF
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import random

from torchvision import transforms
from torchvision import models
import torch.optim as optim



In [3]:
df = pd.read_pickle('combined_df.pkl')

In [4]:
df.shape

(33693, 6)

In [5]:
df = df.dropna(subset=['image'])

In [6]:
df.shape

(29689, 6)

In [7]:
df.columns

Index(['name', 'description', 'label', 'base_name', 'emotion_category',
       'image'],
      dtype='object')

In [8]:
df['label'].unique()

array(['irritation', 'frustration', 'wrath', 'rage', 'anger',
       'exasperation', 'spite', 'fury', 'annoyance', 'resentment',
       'grumpiness', 'outrage', 'aggravation', 'hostility', 'grouchiness',
       'desire', 'passion', 'longing', 'lust', 'infatuation', 'gloom',
       'rejection', 'disappointment', 'unhappiness', 'defeat',
       'agitation', 'insult', 'loneliness', 'alienation', 'hate',
       'displeasure', 'jealousy', 'dislike', 'contempt', 'disgust',
       'envy', 'revulsion', 'bitterness', 'scorn', 'loathing', 'horror',
       'terror', 'dread', 'nervousness', 'fear', 'worry', 'panic',
       'fright', 'hysteria', 'alarm', 'dismay', 'tenseness',
       'apprehension', 'anxiety', 'uneasiness', 'hurt', 'suffering',
       'agony', 'distress', 'anguish', 'insecurity', 'torment',
       'optimism', 'attraction', 'excitement', 'liking', 'eagerness',
       'hope', 'zeal', 'arousal', 'gladness', 'joy', 'happiness',
       'delight', 'rapture', 'cheerfulness', 'joviality', 

In [9]:
# Mapping dictionary
engagement_mapping = {
    "not engaged": [
        "isolation", "neglect", "pity", "sentimentality", "loneliness", "gloom", "alienation", "defeat", "anguish", "dejection",
        "hopelessness", "melancholy", "depression", "homesickness", "longing"
    ],
    "engaged-positive": [
        "lust", "desire", "infatuation", "passion", "attraction", "liking",
        "excitement", "hope", "optimism", "eagerness", "zeal", "arousal", "joy", "zest",
        "cheerfulness", "happiness", "elation", "rapture", "enjoyment", "gladness",
        "bliss", "gaiety", "jubilation", "delight", "euphoria", "jolliness", "joviality",
        "glee", "ecstasy", "caring", "love", "tenderness", "affection", "adoration",
        "fondness", "compassion", "sympathy", "pleasure", "pride", "satisfaction",
        "contentment", "relief", "triumph", "enthusiasm", "amusement", "surprise",
        "astonishment", "amazement", "shock", "thrill", "exhilaration", "enthrallment"
    ],
    "engaged-negative": [
        "irritation", "wrath", "annoyance", "rage", "aggravation", "anger", "resentment",
        "grumpiness", "frustration", "fury", "hostility", "exasperation", "outrage",
        "grouchiness", "spite", "unhappiness", "disappointment", "insult",
        "rejection", "agitation", "bitterness", "hate",
        "disgust", "dislike", "contempt", "scorn", "displeasure", "envy", "loathing",
        "jealousy", "revulsion", "nervousness", "alarm", "fear", "fright", "horror",
        "terror", "dread", "hysteria", "dismay", "apprehension", "worry", "panic",
        "tenseness", "uneasiness", "anxiety", "suffering", "hurt", "agony",
        "insecurity", "distress", "torment", "sadness", "grief", "glumness", "sorrow", "despair", "misery", "woe", "regret", "guilt", "shame", "embarrassment", "mortification",
        "remorse", "humiliation"
    ]
}

# Step 1: Flatten the mapping
flat_mapping = {}
for engagement_type, labels in engagement_mapping.items():
    for label in labels:
        flat_mapping[label] = engagement_type

# Step 2: Apply mapping to your DataFrame
df['engagement_type'] = df['label'].map(flat_mapping)

# Step 3: Optional - check distribution
print(df['engagement_type'].value_counts())




engagement_type
engaged-negative    13160
engaged-positive    12874
not engaged          3652
Name: count, dtype: int64


In [11]:
# Step 1: Define the target sample size
min_class_size = df['engagement_type'].value_counts().min()

# Step 2: Sample each class down to the minimum size
df_balanced = (
    df.groupby('engagement_type', group_keys=False)
      .apply(lambda x: x.sample(n=min_class_size, random_state=42))
      .reset_index(drop=True)
)

# Step 3: Check the balance
print(df_balanced['engagement_type'].value_counts())

engagement_type
engaged-negative    3652
engaged-positive    3652
not engaged         3652
Name: count, dtype: int64


C:\Users\alina\AppData\Local\Temp\ipykernel_12504\3667488993.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_class_size, random_state=42))


In [12]:
df_balanced.columns

Index(['name', 'description', 'label', 'base_name', 'emotion_category',
       'image', 'engagement_type'],
      dtype='object')

In [13]:
df_balanced = df_balanced[['image', 'engagement_type']]

In [14]:
df_balanced['engagement_type'].isna().sum()

0

In [15]:
# --- Data Augmentation for Tensors ---
class TensorAugmentation:
    def __call__(self, img):
        if random.random() > 0.5:
            img = TF.hflip(img)
        if random.random() > 0.5:
            img = TF.rotate(img, angle=random.uniform(-10, 10))
        img = TF.adjust_brightness(img, brightness_factor=random.uniform(0.9, 1.1))
        img = TF.adjust_contrast(img, contrast_factor=random.uniform(0.9, 1.1))
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img = (img - mean) / std
        return img

In [16]:
class EngagementDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.images = list(dataframe['image'].values)
        self.labels = LabelEncoder().fit_transform(dataframe['engagement_type'].values)
        self.labels = torch.tensor(self.labels, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if isinstance(image, torch.Tensor):
            image = image.float()
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

In [17]:
# --- Load Pretrained ResNet18 and Modify ---
def get_resnet18_model(num_classes=3, dropout_p=0.5, freeze_features=True):
    model = models.resnet18(pretrained=True)
    if freeze_features:
        for param in model.parameters():
            param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(dropout_p),
        nn.Linear(512, num_classes)
    )
    return model

In [18]:
class NormalizeOnly:
    def __call__(self, img):
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        return (img - mean) / std

In [19]:
# Split the dataframe (df_balanced is your full, labeled dataset)
train_df, val_df = train_test_split(
    df_balanced, 
    test_size=0.15, 
    stratify=df_balanced["engagement_type"], 
    random_state=42
)

In [20]:
# Define augmentations
train_transform = TensorAugmentation()
val_transform = NormalizeOnly()

In [21]:
# Create datasets
train_dataset = EngagementDataset(train_df, transform=train_transform)
val_dataset = EngagementDataset(val_df, transform=val_transform)

In [22]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [23]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, loss, optimizer
model = get_resnet18_model(num_classes=3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

C:\Users\alina\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\alina\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
# Training loop
for epoch in range(10):  # Adjust number of epochs as needed
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_acc = 100. * correct_val / total_val
    print(f"→ Validation Acc: {val_acc:.2f}%")


Epoch 1, Loss: 1.0922, Train Acc: 39.48%
→ Validation Acc: 32.91%
Epoch 2, Loss: 1.0574, Train Acc: 42.77%
→ Validation Acc: 33.27%
Epoch 3, Loss: 1.0542, Train Acc: 43.80%
→ Validation Acc: 33.03%
Epoch 4, Loss: 1.0428, Train Acc: 44.41%
→ Validation Acc: 32.73%
Epoch 5, Loss: 1.0392, Train Acc: 45.11%
→ Validation Acc: 33.15%
Epoch 6, Loss: 1.0348, Train Acc: 45.47%
→ Validation Acc: 33.27%
Epoch 7, Loss: 1.0320, Train Acc: 45.72%
→ Validation Acc: 33.09%
Epoch 8, Loss: 1.0252, Train Acc: 46.76%
→ Validation Acc: 33.45%
Epoch 9, Loss: 1.0274, Train Acc: 46.49%
→ Validation Acc: 33.21%
Epoch 10, Loss: 1.0275, Train Acc: 47.11%
→ Validation Acc: 33.27%
